In [47]:
!pip install pandas

In [48]:
import pandas as pd
my_file_path = r"C:\Users\user\Desktop\iit\year2 sem1\sdgp\AgroEdge\farming data1.csv"
farming_data=pd.read_csv(my_file_path)
farming_data.head()

,Farmer Name,Farm ID,Farm Size (Acres),Farm Type,Farm Location (Village),Farm Location (District),Crop Type,Brinjal Variety,Planting Start Date,Planting End Date,...,Labor Force (number of workers),Labor Work Schedule (hours/day),Labor Wages (Rs./day),Machinery Used,Machinery Usage Frequency,Market Selling Price (Rs./kg),Total Sales Revenue (Rs.),Profit/Loss (Rs.),Buyer Type,Storage Method
0,Eran Wickramasinghe,6710,2.28,Organic,Hanwella,Colombo,Brinjal,Local,27/10/2024,20/01/2025,...,8,4,2453.45,Machine-2,Monthly,154.11,40680.42,4040.16,Retail,Refrigeration
1,Chandana Fernando,7527,0.70,Mixed,Minuwangoda,Gampaha,Brinjal,Other,07/05/2024,28/08/2024,...,8,8,1399.81,Machine-1,Monthly,100.71,27617.70,8243.59,Retail,Refrigeration
2,Amara Perera,4828,1.14,Conventional,Bulathsinhala,Kalutara,Brinjal,Hybrid,27/07/2024,16/11/2024,...,9,7,2561.95,Machine-3,Monthly,371.97,109366.62,76595.58,Direct Consumers,Refrigeration
3,Chandana Fernando,5629,2.20,Mixed,Pussellawa,Kandy,Brinjal,Hybrid,23/06/2023,04/10/2023,...,5,4,2609.61,Machine-4,Monthly,444.54,87294.32,67207.80,Retail,Refrigeration
4,Dinesh Kumara,5543,2.76,Mixed,Rattota,Matale,Brinjal,Other,14/03/2023,21/05/2023,...,8,9,2025.32,Machine-5,Weekly,313.02,139995.06,111829.22,Wholesale,Refrigeration


In [49]:
print(farming_data.isnull().sum())

Farmer Name                         0
Farm ID                             0
Farm Size (Acres)                   0
Farm Type                           0
Farm Location (Village)             0
Farm Location (District)            0
Crop Type                           0
Brinjal Variety                     0
Planting Start Date                 0
Planting End Date                   0
Expected Harvest (kg)               0
Actual Harvest (kg)                 0
Soil Type                           0
Soil pH Level                       1
Soil Moisture (%)                   1
Seed Type                           0
Seed Variety                        0
Seed Source                         0
Seed Quantity (kg)                  0
Seed Cost (Rs.)                     0
Fertilizer Type                     0
Fertilizer Brand                    0
Fertilizer Quantity (kg/acre)       0
Fertilizer N-P-K Ratio              0
Fertilizer Application Schedule     0
Fertilizer Application Method       0
Fertilizer C

In [50]:
import pandas as pd
# Define the updated function with rounding to two decimal places
def fill_soil_ph(row):
    if pd.isnull(row['Soil pH Level']):  
        # Filter data for the same village
        village_data = farming_data[farming_data['Farm Location (Village)'] == row['Farm Location (Village)']]
        
        # Check if the village has at least 2 records with valid data
        if village_data['Soil pH Level'].count() >= 2:
            village_mean = round(village_data['Soil pH Level'].mean(skipna=True), 2)
            print(f"Filling missing value using village mean ({row['Farm Location (Village)']}): {village_mean}")
            return village_mean
        
        # If no valid data in the village, check the district
        district_data = farming_data[farming_data['Farm Location (District)'] == row['Farm Location (District)']]
        
        # Check if the district has at least 2 records with valid data
        if district_data['Soil pH Level'].count() >= 2:
            district_mean = round(district_data['Soil pH Level'].mean(skipna=True), 2)
            print(f"Filling missing value using district mean ({row['Farm Location (District)']}): {district_mean}")
            return district_mean
        
        # If neither village nor district has at least 2 valid records, return the global mean
        global_mean = round(farming_data['Soil pH Level'].mean(skipna=True), 2)
        print(f"Filling missing value using global mean: {global_mean}")
        return global_mean
    
    return row['Soil pH Level']  

# Apply function to fill missing values
farming_data['Soil pH Level'] = farming_data.apply(fill_soil_ph, axis=1)

# Check if any NaN values remain
missing_values = farming_data['Soil pH Level'].isnull().sum()

# Print the total number of missing values left
print(f"Total missing values remaining: {missing_values}")


Filling missing value using district mean (Kandy): 5.86
Total missing values remaining: 0


In [51]:
print(farming_data.isnull().sum())

Farmer Name                         0
Farm ID                             0
Farm Size (Acres)                   0
Farm Type                           0
Farm Location (Village)             0
Farm Location (District)            0
Crop Type                           0
Brinjal Variety                     0
Planting Start Date                 0
Planting End Date                   0
Expected Harvest (kg)               0
Actual Harvest (kg)                 0
Soil Type                           0
Soil pH Level                       0
Soil Moisture (%)                   1
Seed Type                           0
Seed Variety                        0
Seed Source                         0
Seed Quantity (kg)                  0
Seed Cost (Rs.)                     0
Fertilizer Type                     0
Fertilizer Brand                    0
Fertilizer Quantity (kg/acre)       0
Fertilizer N-P-K Ratio              0
Fertilizer Application Schedule     0
Fertilizer Application Method       0
Fertilizer C

In [52]:
df_kandy = farming_data[farming_data["Farm Location (District)"] == "Kandy"]

# Get the count and sum of pH values
ph_count = df_kandy["Soil pH Level"].count()
ph_sum = df_kandy["Soil pH Level"].sum()
print(ph_count)
print(ph_sum)

4
23.43


In [53]:

# Define the updated function to fill missing Soil Moisture (%) without KNN
def fill_soil_moisture(row):
    if pd.isnull(row['Soil Moisture (%)']):  
        # Filter data for the same village
        village_data = farming_data[farming_data['Farm Location (Village)'] == row['Farm Location (Village)']]
        
        # Check if the village has at least 2 records with valid data
        if village_data['Soil Moisture (%)'].count() >= 2:
            village_mean = round(village_data['Soil Moisture (%)'].mean(skipna=True), 2)
            print(f"Filling missing value using village mean ({row['Farm Location (Village)']}): {village_mean}")
            return village_mean
        
        # If no valid data in the village, check the district
        district_data = farming_data[farming_data['Farm Location (District)'] == row['Farm Location (District)']]
        
        # Check if the district has at least 2 records with valid data
        if district_data['Soil Moisture (%)'].count() >= 2:
            district_mean = round(district_data['Soil Moisture (%)'].mean(skipna=True), 2)
            print(f"Filling missing value using district mean ({row['Farm Location (District)']}): {district_mean}")
            return district_mean
        
        # If neither village nor district has at least 2 valid records, return the global mean
        global_mean = round(farming_data['Soil Moisture (%)'].mean(skipna=True), 2)
        print(f"Filling missing value using global mean: {global_mean}")
        return global_mean
    
    return row['Soil Moisture (%)']  

# Apply function to fill missing values
farming_data['Soil Moisture (%)'] = farming_data.apply(fill_soil_moisture, axis=1)

# Check if any NaN values remain
missing_moisture_values = farming_data['Soil Moisture (%)'].isnull().sum()

# Print the total number of missing values left
print(f"Total missing Soil Moisture (%) values remaining: {missing_moisture_values}")


Filling missing value using district mean (Kandy): 18.89
Total missing Soil Moisture (%) values remaining: 0


In [54]:
df_kandy = farming_data[farming_data["Farm Location (District)"] == "Kandy"]

# Get the count and sum of pH values
moisture_count = df_kandy["Soil Moisture (%)"].count()
moisture_sum = df_kandy["Soil Moisture (%)"].sum()
print(moisture_count)
print(moisture_sum)

4
75.55000000000001


In [55]:
print("Duplicate rows:",farming_data.duplicated().sum())

Duplicate rows: 1


In [56]:
farming_data= farming_data.drop_duplicates()

In [57]:
print("Duplicate rows:",farming_data.duplicated().sum())

Duplicate rows: 0


In [58]:
farming_data.to_csv("cleaned_farmer_data.csv", index=False)


PermissionError: [Errno 13] Permission denied: 'cleaned_farmer_data.csv'

In [59]:
#calculation for to give score by each creteria 
clean_farming_data= pd.read_csv("cleaned_farmer_data.csv")
#calculate yield per acre
df['Yield per Acre'] = df['Actual Harvest (kg)'] / df['Farm Size (Acres)']

#calculate soil suitablity
df['Soil Suitability Score'] = 100 - abs(df['Soil pH Level'] - 6.2) * 20

# 3️⃣ Seed Efficiency
df['Seed Efficiency'] = df['Actual Harvest (kg)'] / df['Seed Quantity (kg)']

# 4️⃣ Fertilizer Efficiency
df['Fertilizer Efficiency'] = df['Actual Harvest (kg)'] / df['Fertilizer Quantity (kg/perche)']

# 5️⃣ Water Efficiency
df['Water Efficiency'] = df['Actual Harvest (kg)'] / df['Water Usage (liters/acre)']

# 6️⃣ Labor Efficiency
df['Labor Efficiency'] = df['Actual Harvest (kg)'] / df['Labor Work Schedule (hours/day)'] 

# 7️⃣ Market Price Score
df['Market Price Score'] = (df['Market Selling Price (Rs./kg)'] / df['Market Avg. Price']) * 100

# 8️⃣ Profitability Score
df['Profitability Score'] = ((df['Total Sales Revenue (Rs.)'] - df['Total Expenses (Rs.)']) / df['Total Expenses (Rs.)']) * 100

# Save the updated dataset with new features
df.to_csv("farmer_data_with_features.csv", index=False)

# Display first few rows to verify calculations
print(df.head())
